In [9]:
%%pyspark
import random
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    current_timestamp, trim, udf, col, lit, row_number, date_format, dayofweek, dayofmonth, dayofyear,
    month, year, weekofyear, when, concat, quarter
)
from pyspark.sql.types import DateType
from pyspark.sql.window import Window
from datetime import datetime, timedelta

SRC_PARQUET_DIR = 'abfss://depi3@depi3.dfs.core.windows.net/Silver Layer'
DIST_PARQUET_DIR = 'abfss://depi3@depi3.dfs.core.windows.net/Gold Layer'

StatementMeta(Depi, 9, 9, Finished, Available, Finished)

In [10]:
accounts_df = spark.read.load(f'{SRC_PARQUET_DIR}/Accounts/part-*.snappy.parquet', format='parquet')
major_df = spark.read.load(f'{SRC_PARQUET_DIR}/Major/part-*.snappy.parquet', format='parquet')
manager_df = spark.read.load(f'{SRC_PARQUET_DIR}/Manager/part-*.snappy.parquet', format='parquet')
office_df = spark.read.load(f'{SRC_PARQUET_DIR}/Office/part-*.snappy.parquet', foramt='parquet')
sales_agent_df = spark.read.load(f'{SRC_PARQUET_DIR}/Sales Agent/part-*.snappy.parquet', format='parquet')
sales_pipeline_df = spark.read.load(f'{SRC_PARQUET_DIR}/Sales PipeLine/part-*.snappy.parquet', format='parquet')
product_df = spark.read.load(f'{SRC_PARQUET_DIR}/Products/part-*.snappy.parquet', format='parquet')
regional_office_df = spark.read.load(f'{SRC_PARQUET_DIR}/Regional Office/part-*.snappy.parquet', format='parquet')
sector_df = spark.read.load(f'{SRC_PARQUET_DIR}/Sector/part-*.snappy.parquet', format='parquet')
series_df = spark.read.load(f'{SRC_PARQUET_DIR}/Series/part-*.snappy.parquet', format='parquet')

StatementMeta(Depi, 9, 10, Finished, Available, Finished)

In [78]:
accounts_df.createOrReplaceTempView("accounts")
major_df.createOrReplaceTempView("major")
manager_df.createOrReplaceTempView("manager")
office_df.createOrReplaceTempView("office")
sales_agent_df.createOrReplaceTempView("sales_agent")
sales_pipeline_df.createOrReplaceTempView("sales_pipeline")
product_df.createOrReplaceTempView("product")
regional_office_df.createOrReplaceTempView("regional_office")
sector_df.createOrReplaceTempView("sector")
series_df.createOrReplaceTempView("series")

StatementMeta(Depi, 9, 82, Finished, Available, Finished)

In [ ]:
def generate_random_date(start_date, end_date):
    """
    Generate a random date between two given dates.

    Parameters:
    start_date (str): The start date in 'YYYY-MM-DD' format.
    end_date (str): The end date in 'YYYY-MM-DD' format.

    Returns:
    datetime: A random date between start_date and end_date.
    """
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    delta = end - start
    random_days = random.randint(0, delta.days)

    return start + timedelta(days=random_days)

generate_random_date_udf = udf(lambda: generate_random_date("2023-01-01", "2023-12-31"), DateType())

# Constructing tables
**Making the DIM tables and the Fact Table**

In [100]:
Dim_Accounts = spark.sql("""
SELECT 
    ROW_NUMBER() OVER (ORDER BY A.AccountID, M.MajorID) AS SrAccountID,
    A.AccountID,
    A.account,
    A.year_established,
    A.revenue,
    A.employees,
    COALESCE(M.MajorName, 'Unknown') Subsidary_of,
    S.SectorName,
    O.OfficeLocation
FROM 
    accounts A
LEFT JOIN 
    major M ON M.MajorID = A.subsidiary_of_ID
LEFT JOIN 
    sector S ON S.SectorID = A.sectorID
LEFT JOIN 
    office O ON O.OfficeID = A.OfficeID;
""")

Dim_Products = spark.sql("""
SELECT
	ROW_NUMBER() OVER (ORDER BY P.ProductID, S.SeriesID) SrProductID ,
	P.ProductID , 
	P.product, 
	P.sales_price , 
	S.SeriesName Series
FROM
	product P
LEFT JOIN 
	series S ON S.SeriesID = P.SeriesID;
""")

Dim_Products = Dim_Products.withColumn("EndDate", generate_random_date_udf())

Dim_Sales_Agent = spark.sql("""
SELECT
	ROW_NUMBER() OVER (ORDER BY S.SalesID, R.RegionalOfficeID) SrAgentID ,
	S.SalesID SalesagentID, 
	S.sales_agent Salesagent,
	M.ManagerName Manager,
	R.RegionalOfficeName RegionalOffice
From
	sales_agent S
LEFT JOIN
	regional_office R
	ON S.regional_officeID = R.RegionalOfficeID
LEFT JOIN
	manager M
	ON S.ManagerID = M.ManagerID;
""")


# this one is for DIM SALES PIPELINE
generate_random_date_udf = udf(lambda: generate_random_date("2020-01-01", "2023-12-31"), DateType())

Dim_Sales_Pipeline = spark.sql("""
SELECT
    ROW_NUMBER() OVER (ORDER BY SalesID, productID, accountID) SrSalesID,
    deal_stage,
    engage_date engage,
    close_date,
    close_value
From
    sales_pipeline;
""")

Dim_Sales_Pipeline = Dim_Sales_Pipeline.withColumn('CreatedAt', generate_random_date_udf())

StatementMeta(Depi, 9, 104, Finished, Available, Finished)

In [27]:
print("DIM Accounts Table:\n", Dim_Accounts.columns)
# Show the query result
display(Dim_Accounts.limit(10))

StatementMeta(Depi, 9, 31, Finished, Available, Finished)

DIM Accounts Table:
 ['SrAccountID', 'AccountID', 'account', 'year_established', 'revenue', 'employees', 'Subsidary_of', 'SectorName', 'OfficeLocation']


SynapseWidget(Synapse.DataFrame, a0ebf806-1fc4-445b-8f4b-b97f0ce37ebd)

In [14]:
# TODO Make the rest of the DIM tables and the Fact table
# TODO Save them in their parquet format in the Gold Layer folder
# TODO After finishing delete all previous data in bronze, silver, and gold layers
# TODO try every thing again to ensure that every this is working smooth

StatementMeta(Depi, 9, 14, Finished, Available, Finished)

In [101]:
Dim_Sales_Agent.columns

StatementMeta(Depi, 9, 105, Finished, Available, Finished)

['SrAgentID', 'SalesagentID', 'Salesagent', 'Manager', 'RegionalOffice']

In [63]:


def random_date(start_date, end_date):
    time_delta = end_date - start_date
    days_between = time_delta.days
    random_days = random.randrange(0, days_between, random.randint(12,34))
    return start_date + timedelta(days=random_days)

# Generate sample date range using datetime
start_date = datetime(2020, 1, 1)
end_date = datetime(2023, 1, 10)
date_list = [random_date(start_date, end_date).strftime("%Y-%m-%d") for x in range(20)]

# Create a DataFrame with just the `date` column initially
df = spark.createDataFrame([(d,) for d in date_list], ["date"])

# Add necessary columns to simulate your SQL query
window_spec = Window.orderBy("date")

Dim_time = (
    df
    .withColumn("timekey", row_number().over(window_spec))
    .withColumn("dayofweek", dayofweek(col("date")))
    .withColumn("dayofmonth", dayofmonth(col("date")))
    .withColumn("dayofyear", dayofyear(col("date")))
    .withColumn("Month", month(col("date")))
    .withColumn("MonthName", date_format(col("date"), "MMMM"))
    .withColumn("quarter", quarter(col("date")))
    .withColumn("Year", year(col("date")))
    .withColumn("WeekofYear", weekofyear(col("date")))
    .withColumn("ISWeekend", when(col("dayofweek").isin(7, 1), lit(True)).otherwise(lit(False)))
    .withColumn("ISHoliday", lit(False))
    .withColumn("HolidayName", lit("UnKnown").cast("string"))
)

# Show the first 10 rows of the resulting DataFrame
display(Dim_time_df.limit(1000))


StatementMeta(Depi, 9, 67, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1f8a57ce-2a3b-453f-8ed4-cbbfb2901bc4)

In [102]:
Dim_Accounts.createOrReplaceTempView("dim_accounts")
Dim_Products.createOrReplaceTempView("dim_products")
Dim_Sales_Agent.createOrReplaceGlobalTempView("dim_sales_agent")
Dim_Sales_Pipeline.createOrReplaceGlobalTempView("dim_sales_pipeline")
Dim_time.createOrReplaceGlobalTempView("dim_time")

StatementMeta(Depi, 9, 106, Finished, Available, Finished)

In [103]:
Fact_Sales = spark.sql("""
SELECT
    ROW_NUMBER() OVER (ORDER BY A.SrAccountID, P.SrProductID, SA.SrAgentID, SP.SrSalesID, T.timekey) FactSalesID,
    A.SrAccountID,
    P.SrProductID,
    SA.SrAgentID,
    SP.SrSalesID,
    T.timekey,
    P.sales_price,
    P.Series,
    A.SectorName,
    A.revenue,
    SP.deal_stage,
    SP.engage,
    SP.close_date,
    SP.close_value
FROM
    dim_accounts A
LEFT JOIN
    dim_products P ON P.SrProductID = A.SrAccountID
LEFT JOIN
    dim_sales_agent SA ON SA.SrAgentID = A.SrAccountID
LEFT JOIN
    dim_sales_pipeline SP ON SP.SrSalesID = A.SrAccountID
LEFT JOIN
    dim_time T ON T.timekey = A.SrAccountID;
""")

display(Fact_Sales.limit(100))

StatementMeta(Depi, 9, 107, Finished, Available, Finished)

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `dim_sales_agent` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 22 pos 4;
'Project [row_number() windowspecdefinition('A.SrAccountID ASC NULLS FIRST, 'P.SrProductID ASC NULLS FIRST, 'SA.SrAgentID ASC NULLS FIRST, 'SP.SrSalesID ASC NULLS FIRST, 'T.timekey ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS FactSalesID#5153, 'A.SrAccountID, 'P.SrProductID, 'SA.SrAgentID, 'SP.SrSalesID, 'T.timekey, 'P.sales_price, 'P.Series, 'A.SectorName, 'A.revenue, 'SP.deal_stage, 'SP.engage, 'SP.close_date, 'SP.close_value]
+- 'Join LeftOuter, ('T.timekey = 'A.SrAccountID)
   :- 'Join LeftOuter, ('SP.SrSalesID = 'A.SrAccountID)
   :  :- 'Join LeftO